In [8]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision

In [24]:
data_dir = 'data/'

# Download and Save MNIST 
data_train = datasets.MNIST(data_dir, train=True, download=True)

random_image = data_train[50000][0]
random_image_label = data_train[50000][1]

# Print the Image using Matplotlib
plt.imshow(random_image)
print("The label of the image is:", random_image_label)

"""
my_transform = transforms.Compose([
 transforms.RandomGrayscale(p=0.5),
 transforms.CenterCrop(100),
 transforms.RandomHorizontalFlip(p=0.5),
 transforms.ToTensor(),
 ])

my_dataset=  torchvision.datasets.ImageFolder(root=data_dir, transform=my_transform)
my_dataloader = torch.utils.data.DataLoader(my_dataset, batch_size=16, shuffle=True, num_workers=4)
"""


IndexError: index 60000 is out of bounds for dimension 0 with size 60000

In [21]:
my_transform = transforms.Compose([
 transforms.ToTensor(),
 ])

data_train = torch.utils.data.DataLoader(
    data_train,
    batch_size=64,
    shuffle=True
    )

In [22]:
torch.cuda.is_available()

False